### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import requests
import subprocess
from bs4 import BeautifulSoup
from contextlib import suppress
import escritoras_latinas.data.load as load
import pandas as pd

### Load data

In [3]:
data_processed = load.data_processed
data_escritoras = load.data_escritoras
assets = load.assets

### Read data

In [4]:
# Read 'csv' file as dataframe
df = pd.read_csv(f'{data_escritoras}')

# Create a list from a column values
urls = df['Url'].tolist()

df.sample(1)

,Nombre,Url,País,Biografía
401,Marjorie Agosín,https://es.wikipedia.org/wiki/Marjorie_Agos%C3...,Chile,Marjorie Agosín (15 de junio de 1955) es una e...


### Web scrapping

In [5]:
# HTML parsing to download images from Wikipedia
for url in urls:
    response = requests.get(url)
    # Return content of the response
    html = response.text
    # Parse html
    soup = BeautifulSoup(html, 'html.parser')
    # Look for <h1> tag with 'firstHeading' id
    title = soup.find('h1', attrs = {'id': 'firstHeading'})
    # Extract text from tag
    text = title.text
    # Process text
    name = text.lower().replace(' ', '_')
    # Look for <a> tag with 'image' class
    imgs = soup.find('a', attrs = {'class':'image'})
    # Handle known exceptions
    with suppress(Exception):
        # Look for <img> tag
        for img in imgs.find_all('img'):
            # Process url
            img = img['src'].replace('//','https://')
            # Ignore url with 'svg' extension
            if re.findall(r'.*\.(svg).*', img):
                continue
            else:
                # Match extension from url    
                extension = re.findall(r'.*\.(jpg|JPG|jpeg|JPEG|png|PNG)$', img)
                # Create file name
                filename = name+'.'+extension[0]
                # Download image using 'wget' command
                cmd = ['wget', '-O', f'{assets}/imgs/{filename}', img]
                # Run the command to download
                subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)